In [86]:
%matplotlib inline
import random
from operator import add
from functools import reduce
import matplotlib.pyplot as plt
import math
import numpy as np
import numpy.random as nrand
import pandas as pd
import itertools
import scipy.stats as stats
import powerlaw
from stockmarket import baselinemodel
from tqdm import tqdm
from pandas_datareader import data
from pylab import plot, show
from math import isclose
from stockmarket.stylizedfacts import *
import itertools
import quandl
from SALib.sample import latin
from statistics import mean

# Calibrate the model using an evolutionary algorithm

In [80]:
SIMTIME = 200
NRUNS = 5
backward_simulated_time = 400
initial_total_money = 26000
init_profit = 1000
init_discount_rate = 0.17

## Parameter space

We define the parameter bounds as follows. 

| Parameter | Values (start, stop, step) |
| -------------| ------------|
| share_chartists       | 0 - 1, 0.1      |
| share_mean_reversion       | 0 - 1, 0.1   |
| order_expiration_time       | 1000 - 10000, 1000      |
| agent_order_price_variability       | 1 - 10, 1      |
| agent_order_variability       | 0.1 - 5       |
| agent_ma_short       | 5 - 100, 5      |
| agent_ma_long       | 50 - 400, 50      |
| agents_hold_thresholds       | 0.0005        |
| Agent_volume_risk_aversion       | 0.1 - 1, 0.1      |
| Agent_propensity_to_switch       | 0.1 - 2.2, 0.1      |
| profit_announcement_working_days       | 5 - 50, 5       |
| price_to_earnings_spread       | 5 - 50, 5       |
| price_to_earnings_heterogeneity       | 5 - 50, 5       |

In [30]:
parameter_space = {'share_chartists':[0.0, 1.0], 'share_mean_reversion':[0.0, 1.0], 'order_expiration_time':[1000, 10000], 
                   'agent_order_price_variability':[1, 10], 'agent_order_variability':[0.1, 5.0], 
                   'agent_ma_short':[5, 100], 'agent_ma_long':[50, 400], 'agents_hold_thresholds':[0.00005,0.01],
                   'agent_volume_risk_aversion':[0.1, 1.0], 'agent_propensity_to_switch':[0.1, 2.2], 
                   'profit_announcement_working_days':[5, 50], 'price_to_earnings_base':[10,20], 
                   'price_to_earnings_heterogeneity':[1.1,2.5], 'price_to_earnings_gap':[4,20],
                   'longMA_heterogeneity':[1.1,1.8], 'shortMA_heterogeneity':[1.1,1.8], 'shortMA_memory_divider':[1, 10]}

Then, we determine the amount of starting points we want for the genetic algorithm and sample the parameter space using a Latin hypercube sample. 

In [69]:
population_size = 100

In [65]:
problem = {
  'num_vars': 17,
  'names': ['share_chartists', 'share_mean_reversion', 'order_expiration_time', 'agent_order_price_variability', 
            'agent_order_variability', 'agent_ma_short', 'agent_ma_long', 'agents_hold_thresholds',
           'agent_volume_risk_aversion', 'agent_propensity_to_switch', 'profit_announcement_working_days',
           'price_to_earnings_base', 'price_to_earnings_heterogeneity', 'price_to_earnings_gap',
           'longMA_heterogeneity', 'shortMA_heterogeneity', 'shortMA_memory_divider'],
  'bounds': [[0, 1], [0, 1], [1000, 10000], [1, 10], 
             [0.1, 5.0], [5, 100], [50, 400], [0.00005,0.01], 
             [0.1, 1], [0.1, 2.2], [5, 50],
             [10,20], [1.1,2.5], [4,20],
             [1.1,1.8], [1.1,1.8], [1, 10]]
}

In [70]:
latin_hyper_cube = latin.sample(problem=problem, N=population_size)
latin_hyper_cube = latin_hyper_cube.tolist()

In [71]:
# transform some of the parameters to integer
for idx, parameters in enumerate(latin_hyper_cube):
    latin_hyper_cube[idx][2] = int(latin_hyper_cube[idx][2])
    latin_hyper_cube[idx][3] = int(latin_hyper_cube[idx][3])
    latin_hyper_cube[idx][4] = int(latin_hyper_cube[idx][4])
    latin_hyper_cube[idx][5] = int(latin_hyper_cube[idx][5])
    latin_hyper_cube[idx][6] = int(latin_hyper_cube[idx][6])
    latin_hyper_cube[idx][10] = int(latin_hyper_cube[idx][10])
    latin_hyper_cube[idx][11] = int(latin_hyper_cube[idx][11])
    latin_hyper_cube[idx][13] = int(latin_hyper_cube[idx][13])
    latin_hyper_cube[idx][16] = int(latin_hyper_cube[idx][16])

## Problem
We try to match average simulation stylized facts as closely as possible to observed stylized facts. 

For that, we use an evolutionary algorithm to minimize a cost function. 

## Create population of individuals 
In our algorithm, an individual is a set a of parameters and its average associated values for stylized facts over several simulation runs. 

In [89]:
# create population
population = []
for parameters in latin_hyper_cube:
    # add an individual to the population
    population.append({'parameters':parameters, 'stylized_facts':[], 'cost':np.inf})

## Define Fitness / cost function
We measure the relative difference between the simulated and actual data using 

$c(s)= \frac{spy(s) - a}{spy(s)}^2$.

Then, for each simulaten, we measure total costs as:

$t(w,v,x,y,z)= c(w) + c(v) + c(x) + c(y) + c(z)
$
where, w represents autocorrelation, v  fat tails, x is clustered volatility, y is long memory, and z is the correlation between price and volume.

In [25]:
def cost_function(observed_values, average_simulated_values):
    """cost function"""
    score = 0
    for obs, sim in zip(observed_values, average_simulated_values):
        score += ((obs - sim) / obs)**2
    return score

In [26]:
def average_fitness(population, observed_values):
    # for every element in the population calculate costs
    for individual in population:
        total_fitness += cost_function(observed_values, individual['average_simulated_values'])
    return total_fitness / (float(len(population)))

## For every individual: simulate the model, score and store the simulation

In [82]:
for idx, individual in tqdm(enumerate(population)):
    parameters = individual['parameters']
    stylized_facts = {'autocorrelation':[],'fat_tails':[],'clustered_volatility':[],'long_memory':[],'vol_vol_clustering':[], 'fitness':[]}
    # name the parameters
    share_chartists= parameters[0]
    share_mean_reversion = parameters[1]
    order_expiration_time = parameters[2]
    agent_order_price_variability = parameters[3]
    agent_order_variability = parameters[4]
    agent_ma_short = parameters[5]
    agent_ma_long = parameters[6]
    agents_hold_thresholds = parameters[7]
    agent_volume_risk_aversion = parameters[8]
    agent_propensity_to_switch = parameters[9]
    profit_announcement_working_days = parameters[10]
    price_to_earnings_base = parameters[11]
    price_to_earnings_heterogeneity = parameters[12]
    price_to_earnings_gap = parameters[13]
    longMA_heterogeneity = parameters[14]
    shortMA_heterogeneity = parameters[15]
    shortMA_memory_divider = parameters[16]
    PE_low_low = price_to_earnings_base
    PE_low_high = int(price_to_earnings_heterogeneity*price_to_earnings_base)
    PE_high_low = PE_low_high + price_to_earnings_gap
    PE_high_high = int(price_to_earnings_heterogeneity*PE_high_low)
    
    # simulate the model 
    for seed in range(NRUNS):
        agents, firms, stocks, order_books = baselinemodel.stockMarketSimulation(seed=seed,
                                                                             simulation_time=1,#SIMTIME,
                                                                         init_backward_simulated_time=int(agent_ma_long*longMA_heterogeneity),
                                                                         number_of_agents=500,
                                                                         share_chartists=share_chartists,
                                                                         share_mean_reversion=share_mean_reversion,
                                                                         amount_of_firms=1,
                                                                         initial_total_money=(initial_total_money,int(initial_total_money*1.1)),
                                                                         initial_profit=(init_profit, init_profit),
                                                                         discount_rate=init_discount_rate,
                                                                         init_price_to_earnings_window=((PE_low_low,
                                                                                                         PE_low_high),
                                                                                                        (PE_high_low,
                                                                                                         PE_high_high)),
                                                                         order_expiration_time=order_expiration_time,
                                                                         agent_order_price_variability=(agent_order_price_variability,agent_order_price_variability),
                                                                         agent_order_variability=agent_order_variability,
                                                                         agent_ma_short=(agent_ma_short, int(agent_ma_short*shortMA_heterogeneity)),
                                                                         agent_ma_long=(agent_ma_long, int(agent_ma_long*longMA_heterogeneity)),
                                                                         agents_hold_thresholds=(1-agents_hold_thresholds, 1+agents_hold_thresholds),
                                                                         agent_volume_risk_aversion=agent_volume_risk_aversion,
                                                                         agent_propensity_to_switch=agent_propensity_to_switch,
                                                                         firm_profit_mu=0.058,
                                                                         firm_profit_delta=0.00396825396,
                                                                         firm_profit_sigma=0.125,
                                                                         profit_announcement_working_days=profit_announcement_working_days,
                                                                             mean_reversion_memory_divider=4,
                                                                         printProgress=False,
                                                                         )
        # store results on autocorrelation etc.
        # test 1
        stylized_facts['autocorrelation'].append(zero_autocorrelation(calculate_returns(order_books[0].transaction_prices_history), 25))
        # test 2
        stylized_facts['fat_tails'].append(fat_tails_kurtosis(calculate_returns(order_books[0].transaction_prices_history)))#fat_tails(calculate_returns(order_books[0].transaction_prices_history))
        # test 3
        stylized_facts['clustered_volatility'].append(clustered_volatility(calculate_returns(order_books[0].transaction_prices_history), 25))
        # test 4
        stylized_facts['long_memory'].append(long_memory(calculate_returns(order_books[0].transaction_prices_history), hurst, 2, 20))
        # TODO add test 5 on volatility / volume correlation 
        
    # add average stylized facts to individual # TODO this will not work because it is randomly shuffled. 
    for key in stylized_facts:
        individual['stylized_facts'].append(mean(stylized_facts[key]))
    # add average fitness to individual TODO 
    individual['fitness'].append(cost_function(observed_values, []))
    
# calculate average fitness
    
    

43it [00:14,  2.93it/s]

no volume
no volume
no volume

100it [00:34,  2.87it/s]

## Evolutionary solution
To solve this problem, we define each parameter space as an **individual** and the collection of all sampled parameters as the **population**. 

In [87]:
mean([1,2,3])
    

2